# Day 6: Sequence Models in Deep Learning

### Exercise 6.1 
Convince yourself a RNN is just an FF unfolded in time. Run the NumpyRNN code. Set break-points and compare with what you learned about back-propagation in the previous day.

Start by loading data Part-of-speech data and configure it for the exercises

### WSJ Data

In [ ]:
# Load Part-of-Speech data 
from lxmls.readers.pos_corpus import PostagCorpusData
data = PostagCorpusData()

Model configuration

In [ ]:
from lxmls.deep_learning.numpy_models.rnn import NumpyRNN

In [ ]:
# RNN configuration
embedding_size = 50   # Size of word embeddings
hidden_size = 20     # size of hidden layer

In [ ]:
model = rnns.NumpyRNN(
    input_size=data.input_size,
    embedding_size=embedding_size,
    hidden_size=hidden_size,
    output_size=data.output_size
)

In [ ]:
train_batches = data.batches('train', batch_size=1)

#### Milestone 1:

Check gradients using the empirical gradient computation

In [ ]:
from lxmls.deep_learning.rnn import rnn_parameter_handlers
from lxmls.deep_learning.numpy_models.rnn import cross_entropy_loss, backpropagation
batch = data.batches('train', batch_size=1)[0]

In [ ]:
gradient = backpropagation(batch['input'], batch['output'], model.parameters)

In [ ]:
print [x.shape for x in model.parameters]

In [ ]:
# Select one parameter from the network
layer_index = 0 # 0
row = 2
column = batch['input'].nonzero()[0][1]

In [ ]:
# Get functions to get and set values of a particular parameter of the model
get_parameter, set_parameter = rnn_parameter_handlers(
    layer_index=layer_index,
    row=row, 
    column=column
)

In [ ]:
import numpy as np
from copy import deepcopy

scale = 10
resolution = 1000
perturbations = np.linspace(-scale, scale, resolution)

# Get study weight value and loss
study_loss = cross_entropy_loss(batch['input'], batch['output'], model.parameters)

# Compute the loss when varying the study weight
parameters = deepcopy(model.parameters)
study_weight = float(get_parameter(parameters))
loss_range = []
for perturbation in perturbations:
    perturbated_parameters = set_parameter(parameters, study_weight + perturbation)
    perturbated_loss = cross_entropy_loss(batch['input'], batch['output'], perturbated_parameters)
    loss_range.append(perturbated_loss)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# Plot empirical
weight_range = study_weight + perturbations
plt.plot(weight_range, loss_range)
plt.plot(study_weight, study_loss, 'xr')
# Plot real
h = plt.plot(
    weight_range,
    get_parameter(gradient)*(weight_range - study_weight) + study_loss, 
    'r--'
)
plt.title('layer %d parameter (%d, %d)' % (layer_index, row, column))
plt.xlabel('parameter value')
plt.ylabel('loss value')